In [1]:
import cv2 # OpenCV
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
import os
import tensorflow_datasets as tfds
import pandas as pd
import torch

In [2]:
print(torch.cuda.is_available())

True


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

path = "D:\FULL"

In [5]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   validation_split=0.5)
training_set = train_datagen.flow_from_directory(path+"/train",
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,
                                                class_mode='categorical',
                                                subset='training')

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True)
validation_set = train_datagen.flow_from_directory(path+"/test",
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,

                                                class_mode='categorical',
                                                subset='training')

Found 15923 images belonging to 7 classes.
Found 3976 images belonging to 7 classes.


In [40]:
resnetv2 = keras.applications.ResNet50V2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
# KALO FULL DIGANTI FALSE 
resnetv2.trainable = True
print(len(resnetv2.layers))

192


In [41]:
i = 0
for layer in resnetv2.layers:
  if i < 170:
    layer.trainable = False
  i+=1

In [42]:
#Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),  # Rotate up to 20 degrees
    tf.keras.layers.RandomTranslation(0.1, 0.15),  # Translate 10% width, 15% height
    tf.keras.layers.RandomBrightness(factor=[0.2, 1.0]),  # Change brightness between 0.2 and 1
    tf.keras.layers.RandomZoom(height_factor=(-0.1, 0.2), width_factor=(-0.1, 0.2)),  # Zoom out up to 10%, zoom in up to 20%
    tf.keras.layers.RandomFlip("horizontal"),  # Flip horizontally
])

input = tf.keras.layers.Input(shape=(224, 224, 3))
# Apply data augmentation to the input
x = data_augmentation(input)
x = resnetv2(input, training=False)
# x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(7, activation = 'softmax')(x)
resnetv2 = tf.keras.models.Model(inputs = input, outputs = x, name = "resnetv2")

resnetv2.summary()

Model: "resnetv2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_27 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50v2 (Functional)         │ (None, 1000)           │    25,613,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 7)              │         7,007 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,620,807 (97.74 MB)

 Trainable params: 9,935,687 (37.90 MB)

 Non-trainable params: 15,685,120 (59.83 MB)

In [43]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=5,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 309s 5s/step - accuracy: 0.2710 - loss: 1.9238 - val_accuracy: 0.2740 - val_loss: 1.8953
Epoch 2/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 306s 5s/step - accuracy: 0.3643 - loss: 1.8615 - val_accuracy: 0.2531 - val_loss: 1.8831
Epoch 3/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 348s 6s/step - accuracy: 0.4185 - loss: 1.8122 - val_accuracy: 0.2740 - val_loss: 1.8499
Epoch 4/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 331s 5s/step - accuracy: 0.4192 - loss: 1.7745 - val_accuracy: 0.2958 - val_loss: 1.8357
Epoch 5/5
62/62 ━━━━━━━━━━━━━━━━━━━━ 12s 129ms/step - accuracy: 0.3906 - loss: 1.7811 - val_accuracy: 0.3382 - val_loss: 1.7969


In [44]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=20,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 305s 5s/step - accuracy: 0.4371 - loss: 1.7337 - val_accuracy: 0.2583 - val_loss: 1.8324
Epoch 2/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 297s 5s/step - accuracy: 0.4210 - loss: 1.7157 - val_accuracy: 0.4083 - val_loss: 1.7102
Epoch 3/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 301s 5s/step - accuracy: 0.4360 - loss: 1.6900 - val_accuracy: 0.4521 - val_loss: 1.6749
Epoch 4/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 298s 5s/step - accuracy: 0.4460 - loss: 1.6619 - val_accuracy: 0.4146 - val_loss: 1.6844
Epoch 5/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 11s 119ms/step - accuracy: 0.3906 - loss: 1.7348 - val_accuracy: 0.4485 - val_loss: 1.6301
Epoch 6/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 301s 5s/step - accuracy: 0.4609 - loss: 1.6301 - val_accuracy: 0.4510 - val_loss: 1.6419
Epoch 7/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 294s 5s/step - accuracy: 0.4485 - loss: 1.6211 - val_accuracy: 0.4427 - val_loss: 1.6371
Epoch 8/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 294s 5s/step - accuracy: 0.4636 - loss: 1.5960 - val_accuracy: 0.4469 -

In [45]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=25,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 318s 5s/step - accuracy: 0.4962 - loss: 1.4484 - val_accuracy: 0.4531 - val_loss: 1.4892
Epoch 2/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 297s 5s/step - accuracy: 0.4944 - loss: 1.4398 - val_accuracy: 0.4938 - val_loss: 1.4579
Epoch 3/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 296s 5s/step - accuracy: 0.5062 - loss: 1.3956 - val_accuracy: 0.4552 - val_loss: 1.5042
Epoch 4/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 295s 5s/step - accuracy: 0.4908 - loss: 1.4122 - val_accuracy: 0.4740 - val_loss: 1.4728
Epoch 5/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 11s 122ms/step - accuracy: 0.6094 - loss: 1.3634 - val_accuracy: 0.5147 - val_loss: 1.4054
Epoch 6/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 297s 5s/step - accuracy: 0.5117 - loss: 1.3768 - val_accuracy: 0.4917 - val_loss: 1.4265
Epoch 7/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 295s 5s/step - accuracy: 0.4981 - loss: 1.3968 - val_accuracy: 0.5240 - val_loss: 1.3583
Epoch 8/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 293s 5s/step - accuracy: 0.5322 - loss: 1.3122 - val_accuracy: 0.4833 -

In [46]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=15,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 379s 6s/step - accuracy: 0.5638 - loss: 1.2017 - val_accuracy: 0.5156 - val_loss: 1.3554
Epoch 2/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 351s 6s/step - accuracy: 0.5964 - loss: 1.1719 - val_accuracy: 0.5333 - val_loss: 1.3199
Epoch 3/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 358s 6s/step - accuracy: 0.5723 - loss: 1.1980 - val_accuracy: 0.5531 - val_loss: 1.2946
Epoch 4/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 355s 6s/step - accuracy: 0.6024 - loss: 1.1436 - val_accuracy: 0.5490 - val_loss: 1.2805
Epoch 5/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 14s 152ms/step - accuracy: 0.5000 - loss: 1.2136 - val_accuracy: 0.5221 - val_loss: 1.2886
Epoch 6/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 310s 5s/step - accuracy: 0.5904 - loss: 1.1612 - val_accuracy: 0.5229 - val_loss: 1.3015
Epoch 7/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 294s 5s/step - accuracy: 0.6017 - loss: 1.1273 - val_accuracy: 0.5219 - val_loss: 1.2782
Epoch 8/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 286s 5s/step - accuracy: 0.6012 - loss: 1.1286 - val_accuracy: 0.5521 -

In [47]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=20,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 426s 7s/step - accuracy: 0.6567 - loss: 1.0394 - val_accuracy: 0.5323 - val_loss: 1.2811
Epoch 2/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 390s 6s/step - accuracy: 0.6352 - loss: 1.0490 - val_accuracy: 0.5562 - val_loss: 1.2689
Epoch 3/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 398s 6s/step - accuracy: 0.6376 - loss: 1.0393 - val_accuracy: 0.5292 - val_loss: 1.3115
Epoch 4/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 388s 6s/step - accuracy: 0.6282 - loss: 1.0447 - val_accuracy: 0.5229 - val_loss: 1.2800
Epoch 5/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 14s 148ms/step - accuracy: 0.7031 - loss: 0.9027 - val_accuracy: 0.5147 - val_loss: 1.3547
Epoch 6/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 378s 6s/step - accuracy: 0.6502 - loss: 1.0329 - val_accuracy: 0.5354 - val_loss: 1.2698
Epoch 7/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 378s 6s/step - accuracy: 0.6745 - loss: 0.9756 - val_accuracy: 0.5271 - val_loss: 1.2989
Epoch 8/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 374s 6s/step - accuracy: 0.6641 - loss: 0.9851 - val_accuracy: 0.5490 -

In [48]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=15,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 381s 6s/step - accuracy: 0.7078 - loss: 0.8884 - val_accuracy: 0.5677 - val_loss: 1.2555
Epoch 2/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 369s 6s/step - accuracy: 0.7035 - loss: 0.8841 - val_accuracy: 0.5323 - val_loss: 1.3247
Epoch 3/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 367s 6s/step - accuracy: 0.7088 - loss: 0.8660 - val_accuracy: 0.5542 - val_loss: 1.2798
Epoch 4/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 355s 6s/step - accuracy: 0.7205 - loss: 0.8647 - val_accuracy: 0.5906 - val_loss: 1.1759
Epoch 5/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 13s 140ms/step - accuracy: 0.7188 - loss: 0.8046 - val_accuracy: 0.5735 - val_loss: 1.2425
Epoch 6/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 350s 6s/step - accuracy: 0.7281 - loss: 0.8137 - val_accuracy: 0.5490 - val_loss: 1.3144
Epoch 7/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 349s 6s/step - accuracy: 0.7248 - loss: 0.8574 - val_accuracy: 0.5448 - val_loss: 1.3274
Epoch 8/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 349s 6s/step - accuracy: 0.7424 - loss: 0.7925 - val_accuracy: 0.5604 -

In [49]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=15,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 370s 6s/step - accuracy: 0.7560 - loss: 0.7567 - val_accuracy: 0.5615 - val_loss: 1.3048
Epoch 2/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 349s 6s/step - accuracy: 0.7357 - loss: 0.7954 - val_accuracy: 0.5729 - val_loss: 1.2790
Epoch 3/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 348s 6s/step - accuracy: 0.7567 - loss: 0.7619 - val_accuracy: 0.5604 - val_loss: 1.3592
Epoch 4/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 344s 6s/step - accuracy: 0.7587 - loss: 0.7454 - val_accuracy: 0.5698 - val_loss: 1.2730
Epoch 5/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 13s 132ms/step - accuracy: 0.7656 - loss: 0.7419 - val_accuracy: 0.4853 - val_loss: 1.4959
Epoch 6/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 346s 6s/step - accuracy: 0.7647 - loss: 0.7257 - val_accuracy: 0.5677 - val_loss: 1.3024
Epoch 7/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 345s 6s/step - accuracy: 0.7577 - loss: 0.7593 - val_accuracy: 0.5531 - val_loss: 1.3067
Epoch 8/15
62/62 ━━━━━━━━━━━━━━━━━━━━ 343s 6s/step - accuracy: 0.7690 - loss: 0.7380 - val_accuracy: 0.5667 -

In [50]:
resnetv2.save("resnetv2-krg-1.keras")

In [51]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=25,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 331s 5s/step - accuracy: 0.7838 - loss: 0.6884 - val_accuracy: 0.5500 - val_loss: 1.3498
Epoch 2/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 320s 5s/step - accuracy: 0.7875 - loss: 0.6907 - val_accuracy: 0.5250 - val_loss: 1.4720
Epoch 3/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 315s 5s/step - accuracy: 0.7886 - loss: 0.6924 - val_accuracy: 0.5792 - val_loss: 1.3355
Epoch 4/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 313s 5s/step - accuracy: 0.7885 - loss: 0.6732 - val_accuracy: 0.5729 - val_loss: 1.3568
Epoch 5/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 12s 126ms/step - accuracy: 0.7812 - loss: 0.6745 - val_accuracy: 0.5662 - val_loss: 1.3609
Epoch 6/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 313s 5s/step - accuracy: 0.8139 - loss: 0.6050 - val_accuracy: 0.5500 - val_loss: 1.3809
Epoch 7/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 312s 5s/step - accuracy: 0.8086 - loss: 0.6143 - val_accuracy: 0.5469 - val_loss: 1.3840
Epoch 8/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 313s 5s/step - accuracy: 0.8046 - loss: 0.6244 - val_accuracy: 0.5469 -

In [54]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

resnetv2.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = resnetv2.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=10,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 360s 6s/step - accuracy: 0.8654 - loss: 0.4625 - val_accuracy: 0.5542 - val_loss: 1.5622
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 334s 5s/step - accuracy: 0.8833 - loss: 0.4260 - val_accuracy: 0.5698 - val_loss: 1.4389
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 330s 5s/step - accuracy: 0.8795 - loss: 0.4237 - val_accuracy: 0.5635 - val_loss: 1.4712
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 330s 5s/step - accuracy: 0.8691 - loss: 0.4493 - val_accuracy: 0.5615 - val_loss: 1.5015
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 13s 134ms/step - accuracy: 0.8906 - loss: 0.4342 - val_accuracy: 0.5515 - val_loss: 1.5021
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 335s 5s/step - accuracy: 0.8759 - loss: 0.4409 - val_accuracy: 0.5406 - val_loss: 1.4903
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 334s 5s/step - accuracy: 0.8883 - loss: 0.3980 - val_accuracy: 0.5760 - val_loss: 1.3980
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 333s 5s/step - accuracy: 0.9004 - loss: 0.3715 - val_accuracy: 0.5437 -

In [ ]:
resnetv2.save("resnetv2-krgg-170.keras")

In [4]:
new_resnetv2_krg = tf.keras.models.load_model("resnetv2-krg-1.keras")

In [5]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

new_resnetv2_krg.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = new_resnetv2_krg.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=10,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/10


c:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


62/62 ━━━━━━━━━━━━━━━━━━━━ 402s 6s/step - accuracy: 0.7868 - loss: 0.6917 - val_accuracy: 0.6000 - val_loss: 1.2589
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 370s 6s/step - accuracy: 0.7894 - loss: 0.6757 - val_accuracy: 0.5594 - val_loss: 1.3294
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 376s 6s/step - accuracy: 0.8008 - loss: 0.6506 - val_accuracy: 0.5573 - val_loss: 1.3606
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 361s 6s/step - accuracy: 0.8028 - loss: 0.6528 - val_accuracy: 0.5615 - val_loss: 1.3501
Epoch 5/10
 1/62 ━━━━━━━━━━━━━━━━━━━━ 5:14 5s/step - accuracy: 0.8125 - loss: 0.6064

c:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


62/62 ━━━━━━━━━━━━━━━━━━━━ 15s 157ms/step - accuracy: 0.8125 - loss: 0.6064 - val_accuracy: 0.5588 - val_loss: 1.3411
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 354s 6s/step - accuracy: 0.7998 - loss: 0.6379 - val_accuracy: 0.5656 - val_loss: 1.3176
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 356s 6s/step - accuracy: 0.8150 - loss: 0.6077 - val_accuracy: 0.5490 - val_loss: 1.4277
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 335s 5s/step - accuracy: 0.8071 - loss: 0.6473 - val_accuracy: 0.5531 - val_loss: 1.3549
Epoch 9/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 328s 5s/step - accuracy: 0.8153 - loss: 0.6132 - val_accuracy: 0.5531 - val_loss: 1.3818
Epoch 10/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 13s 136ms/step - accuracy: 0.7812 - loss: 0.8092 - val_accuracy: 0.6250 - val_loss: 1.2019


In [6]:
steps_per_epoch = training_set.n // 256
validation_steps = validation_set.n // 256

new_resnetv2_krg.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = new_resnetv2_krg.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=45,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 370s 6s/step - accuracy: 0.7969 - loss: 0.6333 - val_accuracy: 0.5437 - val_loss: 1.4199
Epoch 2/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 354s 6s/step - accuracy: 0.8232 - loss: 0.5902 - val_accuracy: 0.5604 - val_loss: 1.3631
Epoch 3/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 343s 6s/step - accuracy: 0.8259 - loss: 0.5748 - val_accuracy: 0.5719 - val_loss: 1.3338
Epoch 4/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 342s 6s/step - accuracy: 0.8282 - loss: 0.5743 - val_accuracy: 0.5521 - val_loss: 1.4228
Epoch 5/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 13s 130ms/step - accuracy: 0.7656 - loss: 0.7020 - val_accuracy: 0.6397 - val_loss: 1.1352
Epoch 6/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 357s 6s/step - accuracy: 0.8366 - loss: 0.5643 - val_accuracy: 0.5490 - val_loss: 1.3635
Epoch 7/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 356s 6s/step - accuracy: 0.8387 - loss: 0.5552 - val_accuracy: 0.5594 - val_loss: 1.4107
Epoch 8/45
62/62 ━━━━━━━━━━━━━━━━━━━━ 353s 6s/step - accuracy: 0.8257 - loss: 0.5788 - val_accuracy: 0.5510 -

In [7]:
new_resnetv2_krg.save("resnetv2-krg-170.keras")

VGG19

In [6]:
VGG19Train_dikit_new = tf.keras.models.load_model("C:/Users/ACER/Downloads/VGG19Augmented_TrainableDikit6/VGG19Augmented_TrainableDikit6.keras")

In [7]:
steps_per_epoch = training_set.n // 128
validation_steps = validation_set.n // 128

VGG19Train_dikit_new.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = VGG19Train_dikit_new.fit(x=training_set,
                    epochs=1,
                    steps_per_epoch=steps_per_epoch
                    )

c:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


124/124 ━━━━━━━━━━━━━━━━━━━━ 2820s 23s/step - accuracy: 0.7137 - loss: 0.7779


In [8]:
steps_per_epoch = training_set.n // 128
validation_steps = validation_set.n // 128

VGG19Train_dikit_new.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = VGG19Train_dikit_new.fit(x=training_set,
                    epochs=4,
                    steps_per_epoch=steps_per_epoch
                    )

Epoch 1/4
124/124 ━━━━━━━━━━━━━━━━━━━━ 3081s 25s/step - accuracy: 0.7030 - loss: 0.8059
Epoch 2/4
124/124 ━━━━━━━━━━━━━━━━━━━━ 4417s 36s/step - accuracy: 0.7172 - loss: 0.7685
Epoch 3/4
124/124 ━━━━━━━━━━━━━━━━━━━━ 44s 6ms/step - accuracy: 0.6719 - loss: 0.7991    
Epoch 4/4


c:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


124/124 ━━━━━━━━━━━━━━━━━━━━ 4091s 33s/step - accuracy: 0.7297 - loss: 0.7301


In [9]:
steps_per_epoch = training_set.n // 128
validation_steps = validation_set.n // 128

VGG19Train_dikit_new.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = VGG19Train_dikit_new.fit(x=training_set,
                    epochs=5,
                    steps_per_epoch=steps_per_epoch
                    )

Epoch 1/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 2924s 23s/step - accuracy: 0.7294 - loss: 0.7361
Epoch 2/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 2884s 23s/step - accuracy: 0.7386 - loss: 0.7207
Epoch 3/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.7255 - loss: 1.0132  
Epoch 4/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 3006s 24s/step - accuracy: 0.7544 - loss: 0.6850
Epoch 5/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 2872s 23s/step - accuracy: 0.7513 - loss: 0.6693


In [10]:
steps_per_epoch = training_set.n // 128
validation_steps = validation_set.n // 128

VGG19Train_dikit_new.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = VGG19Train_dikit_new.fit(x=training_set,
                    epochs=17,
                    steps_per_epoch=steps_per_epoch
                    )

Epoch 1/17
124/124 ━━━━━━━━━━━━━━━━━━━━ 2731s 22s/step - accuracy: 0.7322 - loss: 0.7248
Epoch 2/17
124/124 ━━━━━━━━━━━━━━━━━━━━ 2873s 23s/step - accuracy: 0.7399 - loss: 0.7126
Epoch 3/17
124/124 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.7031 - loss: 0.8141  
Epoch 4/17
124/124 ━━━━━━━━━━━━━━━━━━━━ 3093s 25s/step - accuracy: 0.7741 - loss: 0.6347
Epoch 5/17
124/124 ━━━━━━━━━━━━━━━━━━━━ 3262s 26s/step - accuracy: 0.7561 - loss: 0.6521
Epoch 6/17
124/124 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.7344 - loss: 0.7084  
Epoch 7/17
124/124 ━━━━━━━━━━━━━━━━━━━━ 2651s 21s/step - accuracy: 0.7856 - loss: 0.5953
Epoch 8/17
124/124 ━━━━━━━━━━━━━━━━━━━━ 2719s 22s/step - accuracy: 0.7714 - loss: 0.6355
Epoch 9/17
124/124 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.6875 - loss: 0.7189  
Epoch 10/17
124/124 ━━━━━━━━━━━━━━━━━━━━ 2809s 23s/step - accuracy: 0.7963 - loss: 0.5772
Epoch 11/17
124/124 ━━━━━━━━━━━━━━━━━━━━ 2724s 22s/step - accuracy: 0.7795 - loss: 0.5939
Epoch 12/17
124/124

In [11]:
VGG19Train_dikit_new.save("VGG19_TrainableDikit_87.keras")